In [1]:
import cv2
import os
import numpy as np

# Initialize LBPH face recognizer
recognizer = cv2.face_LBPHFaceRecognizer.create()

# Create a function to get the images and labels from the dataset
def get_images_and_labels(dataset_path):
    face_images = []
    labels = []
    
    # Create a mapping from person names to unique integer labels
    label_map = {}
    current_label = 0

    for person_folder in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_folder)
        
        # Check if it's a directory
        if os.path.isdir(person_path):
            label_map[person_folder] = current_label
            
            for image_filename in os.listdir(person_path):
                image_path = os.path.join(person_path, image_filename)
                face_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                face_images.append(face_img)
                labels.append(current_label)
                
            current_label += 1

    return face_images, labels, label_map

# Train the recognizer
dataset_path = 'C:\\Users\\91895\\Downloads\\dataset'  # Replace with your actual dataset path
face_images, labels, label_map = get_images_and_labels(dataset_path)

# Train the recognizer with integer labels
recognizer.train(face_images, np.array(labels))
recognizer.save('trained_model.yml')


In [2]:
import cv2
import numpy as np

# Load the trained face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trained_model.yml')

# Load the face detection cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Create a dictionary of person IDs and their corresponding names
id_to_name = {0: "Person 1", 1: "Person 2", 2: "Person 3"}  # Update with your own dataset

# Initialize the webcam (change the index to 0 for built-in camera or the camera index you have)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw a rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Recognize the face
        id_, confidence = recognizer.predict(gray[y:y + h, x:x + w])

        if confidence < 70:  # You can adjust the confidence threshold
            name = id_to_name[id_]
        else:
            name = "Unknown"

        # Display the name and confidence level
        cv2.putText(frame, f"{name} {confidence:.2f}%", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow('Face Recognition', frame)

    if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
        break

cap.release()
cv2.destroyAllWindows()
